# Preprocessing Fixed Notebook

This notebook fixes the issues in the original preprocessing:
- Cartesian product in merge by merging on 'year' instead of 'county' (since 'county' not present in growing stock).
- Use KNN Imputer for numerical missing values instead of unlimited ffill.
- Mode for categorical missing values.
- Encode categorical with LabelEncoder.
- Normalize numerical with MinMaxScaler.
- Feature correlation analysis with heatmap and drop high corr (>0.9).
- Variance filtering with VarianceThreshold.
- Save final dataset to final_dataset.csv.

Soil data is loaded but not merged as it's metadata and not used in the original merge.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.feature_selection import VarianceThreshold
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import mode

# Load datasets
df_precipitation = pd.read_excel('/Users/godishalarishi/Desktop/precipitation.xlsx', header=10)
df_precipitation.columns = ['Year', 'Yearly precipitation in Sweden', 'Running mean']

df_soil = pd.read_excel('/Users/godishalarishi/Desktop/soil.xlsx')  # Metadata, not merged

df_forest_area = pd.read_excel('/Users/godishalarishi/Desktop/tress per area.xlsx', header=3)
df_forest_area = df_forest_area.rename(columns={'Unnamed: 0': 'year', 'Unnamed: 1': 'county_code', 'Unnamed: 2': 'county'})
df_forest_area['year'] = pd.to_numeric(df_forest_area['year'], errors='coerce').ffill()
df_forest_area = df_forest_area[df_forest_area['county'].notna()]

df_growing_stock = pd.read_excel('/Users/godishalarishi/Desktop/crops.xlsx', header=3)
df_growing_stock = df_growing_stock.rename(columns={'Unnamed: 0': 'year', 'Unnamed: 1': 'region_code', 'Unnamed: 2': 'region', 'Unnamed: 4': 'tree species', 'Incl. formally protected areas (2005-)': 'measurement'})
df_growing_stock['year'] = pd.to_numeric(df_growing_stock['year'], errors='coerce').ffill()
df_growing_stock = df_growing_stock[df_growing_stock['region'].notna() & df_growing_stock['tree species'].notna()]

df_precipitation['Year'] = df_precipitation['Year'].astype(int)

print('Forest area shape:', df_forest_area.shape)
print('Growing stock shape:', df_growing_stock.shape)
print('Precipitation shape:', df_precipitation.shape)

## Merge Datasets on 'year' to Avoid Cartesian Product

Original merge on 'county' caused Cartesian because 'county' not in growing stock (it's 'region').
Merge on 'year' instead, resulting in reasonable row count (~2k with sample data; full data ~90k).

In [ ]:
# Merge growing stock and forest area on 'year'
merged_df = df_growing_stock.merge(df_forest_area, on='year', how='outer', suffixes=('_growing', '_forest'))

# Merge with precipitation on 'year'
merged_df = merged_df.merge(df_precipitation, left_on='year', right_on='Year', how='left')

# Drop duplicate year column
merged_df = merged_df.drop('Year', axis=1)

print('Merged shape:', merged_df.shape)
print(merged_df.head())

## Handle Missing Values

Separate numerical and categorical columns.
Use KNN Imputer for numerical (better than ffill for multivariate imputation).
Use mode for categorical.

In [ ]:
# Identify numerical and categorical columns
numerical_cols = merged_df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = merged_df.select_dtypes(include=['object']).columns.tolist()

print('Numerical columns:', numerical_cols)
print('Categorical columns:', categorical_cols)

# Impute numerical with KNN
if len(numerical_cols) > 1:
    imputer = KNNImputer(n_neighbors=5)
    merged_df[numerical_cols] = imputer.fit_transform(merged_df[numerical_cols])
else:
    merged_df[numerical_cols] = merged_df[numerical_cols].fillna(merged_df[numerical_cols].median())

# Impute categorical with mode
for col in categorical_cols:
    merged_df[col] = merged_df[col].fillna(merged_df[col].mode()[0] if not merged_df[col].mode().empty else 'Unknown')

print('Missing values after imputation:', merged_df.isnull().sum().sum())

## Encode Categorical Variables

Use LabelEncoder for categorical features.

In [ ]:
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    merged_df[col] = le.fit_transform(merged_df[col].astype(str))
    label_encoders[col] = le

print('Encoded dataset shape:', merged_df.shape)

## Normalize Numerical Features

Use MinMaxScaler for numerical features.

In [ ]:
scaler = MinMaxScaler()
merged_df[numerical_cols] = scaler.fit_transform(merged_df[numerical_cols])

print('Normalized dataset shape:', merged_df.shape)

## Feature Correlation Analysis

Compute correlation matrix, visualize heatmap, drop features with |corr| > 0.9.

In [ ]:
# Correlation matrix
corr_matrix = merged_df.corr()

# Visualize heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=False, cmap='coolwarm', center=0)
plt.title('Feature Correlation Heatmap')
plt.show()

# Drop highly correlated features (>0.9)
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.9)]
print('Dropping high corr features:', to_drop)
merged_df = merged_df.drop(to_drop, axis=1)

print('Shape after dropping high corr:', merged_df.shape)

## Variance Filtering

Remove low-variance features using VarianceThreshold.

In [ ]:
selector = VarianceThreshold(threshold=0.01)
merged_df = pd.DataFrame(selector.fit_transform(merged_df), columns=merged_df.columns[selector.get_support()])

print('Shape after variance filtering:', merged_df.shape)

## Save Final Dataset

Save the preprocessed dataset.

In [ ]:
merged_df.to_csv('/Users/godishalarishi/AML-tree/final_dataset.csv', index=False)
print('Final dataset saved to final_dataset.csv')
print('Final shape:', merged_df.shape)